In [1]:
import tensorflow as tf
from tensorflow.keras.layers  import Input,Conv2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from PIL import Image 
from numpy import rot90
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import SGD,Adadelta
from tensorflow.keras.losses import CategoricalCrossentropy
import matplotlib.pyplot as plt


In [3]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()
# img_list = []
# y = []
# augmenting images by rotation
# for index in range(len(x_train)):
#     array = x_train[index]
#     for i in range(2):
#         array = rot90(array) # rotating an image by 90
#         img_list.append(array) # append image to training data
#         y.append(y_train[index])

# y = np.array(y)
# img_list = np.array(img_list)
# x_train = np.concatenate((x_train,img_list),axis = 0)
# y_train = np.concatenate((y_train,y),axis = 0)
# rescale images : dividing by 255

x_train = x_train/255
x_test = x_test/255

# one hot encode labels
label_binarizer = LabelBinarizer()
label_binarizer.fit(range(10)) # ranging 0 to 9
y_train = label_binarizer.transform(y_train)
y_test = label_binarizer.transform(y_test)
y_train.shape,y_test.shape

170500096/170498071 [==============================] - 35s 0us/step


((50000, 10), (10000, 10))

In [5]:
# neural networks implementation
# neural network A
class NetA(Model):
    def __init__(self):
        super(NetA,self).__init__()
        self.inputs = Input((32,32,3),name="input_layer")
        self.flatten_layer = Flatten()(self.inputs) # Flatten Layer
        self.outputs = Dense(units=10,activation="softmax")(self.flatten_layer)
        self.net_a = Model(inputs=self.inputs,outputs=self.outputs,name="NET_A")
    def call(self,inputs):
        return self.net_a(inputs)



# neural network B
class NetB(Model):
    def __init__(self):
        super(NetB,self).__init__()
        self.inputs = Input((32,32,3),name="input_layer")
        self.flatten_layer = Flatten()(self.inputs) # Flatten Layer
        self.hidden_dense = Dense(units=300,name="hidden_dense_layer",activation="relu")(self.flatten_layer)# Hidden Dense Layer
        self.outputs = Dense(units=10,activation="softmax")(self.hidden_dense) 
        self.net_b = Model(inputs=self.inputs,outputs=self.outputs,name="NET_B")
    def call(self,inputs):
        return self.net_b(inputs)



# neural network C(Convolutional Neural Network)
class NetC(Model):
    def __init__(self):
        super(NetC,self).__init__()
        self.inputs = Input((32,32,3),name="input_layer")
        self.cnn_1 = Conv2D(filters=25,kernel_size=(5,5),activation="relu")(self.inputs) # Convolutional Layer with 25 filters and (5,5) kernel_size
        self.max_pool_1 = MaxPooling2D(pool_size=(2,2),strides=2)(self.cnn_1) # MaxPooling layer with size (2,2)
        self.flatten_layer = Flatten()(self.max_pool_1) # Flatten Layer
        self.outputs = Dense(units=10,activation="softmax")(self.flatten_layer)
        self.net_c = Model(inputs=self.inputs,outputs=self.outputs,name="NET_C")
    def call(self,inputs):
        return self.net_c(inputs)

In [6]:
def plot_history(histories):
    for history in histories:
        print( "-"*25+ history["name"] + "-"*25 + "\n")
        plt.plot(range(50),history["train_accs"],color="orange",label="Train Acc")
        plt.plot(range(50),history["test_accs"],color="blue",label="Validation Acc")
        plt.legend(loc= "upper center")
        plt.show()
        print(f"Training Loss : {history['train_loss'][-1]} Validation Loss : {history['test_loss'][-1]} ")
        print(f"Training Accuracy : {history['train_accs'][-1]} Validation Accuracy : {history['test_accs'][-1]}" )

In [7]:
nets = [NetA(),NetB(),NetC()]
histories = []
for net in nets:
    print("-"*25 + f"Training {type(net).__name__}" + "-"*25)
    net.compile(loss = CategoricalCrossentropy(),optimizer = SGD(learning_rate=0.01),metrics=["acc"]) # compile the neural network
    history = net.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=50) # fit the model
    histories.append({
        "name": type(net).__name__,
        "net" : net,
        "train_accs" : history.history["acc"],
        "test_accs" : history.history["val_acc"],
        "train_loss" : history.history["loss"],
        "test_loss"  : history.history["val_loss"]
    })
plot_history(histories)
